<a href="https://colab.research.google.com/github/jakobengdahl/SCBDatahubDefinition/blob/main/DataEkosystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**För att testa lösningen:**
1.   Klicka på knappen: Open in Colab
2.   Logga in i colab om du behöver det
3.   Klicka på menyn: Körning -> Kör Alla (och välj "Kör ändå")
4.   När alla rader frått “en grön bock”, gå längst ner på sidan och klicka på formuläret

In [1]:
#@title
!pip install networkx --quiet
!pip install yfiles_jupyter_graphs --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.3 MB/s eta 0:00:00


In [2]:
#@title
# Import required libraries
from google.colab import widgets
from google.colab import output
from google.colab import data_table
import ipywidgets as widgets2
#from IPython.display import display
from ipywidgets import HTML, VBox, HBox, Tab, Label, Button, Text, Textarea
import json
import pandas as pd
import numpy as np
import markdown
from networkx import florentine_families_graph
from yfiles_jupyter_graphs import GraphWidget
from typing import Dict
from ipywidgets import Layout
from google.colab import files
data_table.enable_dataframe_formatter()

In [3]:
#@title
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
#@title
def setup_ui(df):
    out = widgets2.Output()
    with out:
        display(df)
    return out

def on_search_click(btn):
  url2 = 'https://raw.githubusercontent.com/jakobengdahl/SCBDatahubDefinition/main/ai_search.csv'
  df_ai = pd.read_csv(url2)
  df_ai = df_ai.replace(np.nan, '-')
  ai_tabell = setup_ui(df_ai)
  ai.children=tuple(list(ai.children) + [ai_tabell])

url = 'https://raw.githubusercontent.com/jakobengdahl/SCBDatahubDefinition/main/artefakt.csv'
df = pd.read_csv(url)
df = df.replace(np.nan, '-')

w = GraphWidget()
w.nodes = [
    {"id": 0, "properties": {"label": "Skatteverket", "url": "https://www.skatteverket.se"}},
    {"id": 1, "properties": {"label": "Bolagsverket", "url": "https://www.bolagsverket.se"}}
]

w.edges = [
    {"id": 0, "start": 0, "end": 1, "properties": {"label": "collaboration"}}
]

In [5]:
#@title
# Initialize the datahubdefinition_data dictionary
datahubdefinition_data = {
    "datahubdefinition": {
        "name": "",
        "organizational_affiliation": "",
        "connected_datahubs": [],
        "artifacts": []
    }
}

# Function to add a new connected datahub
def add_connected_datahub(button):
    connected_datahub = {
        "name": benamning_input2.value,
        "definition_path": sokvag_till_definition_input.value,
        "more_info_path": sokvag_till_mer_info_input.value
    }
    
    datahubdefinition_data["datahubdefinition"]["connected_datahubs"].append(connected_datahub)
    
    # Create a label for the new connected datahub and add it to the connected_datahub_labels VBox
    new_label = Label(connected_datahub["name"])
    connected_datahub_labels.children += (new_label,)


# Function to add a new artifact
def add_artifact(button):
    artifact = {
        "name": benamning_input3.value,
        "description": beskrivning_input.value,
        "license": licens_input.value,
        "accessibility_level": tillganglighetsgrad_input.value,
        "contact_path": kontaktvag_input.value,
        "keywords": nyckelord_input.value,
        "created_date": skapat_datum_input.value,
        "dependencies": beroenden_input.value,
        "path": sokvag_input.value
    }
    
    datahubdefinition_data["datahubdefinition"]["artifacts"].append(artifact)
    
    # Create a label for the new artifact and add it to the artifact_labels VBox
    new_label = Label(artifact["name"])
    artifact_labels.children += (new_label,)

# Define the save_datadefinition function
def save_datadefinition(button):
    # Read data from form1 and update the JSON structure
    datahubdefinition_data["datahubdefinition"]["name"] = benamning_input1.value
    datahubdefinition_data["datahubdefinition"]["organizational_affiliation"] = organisatorisk_tillhorighet_input.value

    # Save the JSON structure to a file called "DatahubDefinition.json"
    with open("DatahubDefinition.json", "w") as outfile:
        json.dump(datahubdefinition_data, outfile, indent=2)

    # Create a download link for the JSON file using Google Colab
    files.download('DatahubDefinition.json')

    # Update the buttons HBox to include the download link
    #buttons.children = (spara_button)

    print("Data definition saved.")









In [6]:
#@title
main_tb = widgets.TabBar(['Start', 'Skapa datahubbdefinition', 'Dataekosystem', 'Hitta artefakter'])

with main_tb.output_to(3):
  hitta_tb = widgets.TabBar(['Hitta artefakter', 'AI matching'])
  with hitta_tb.output_to(1):
    ai_headline = HTML(value="<h2>AI matching</h2>")
    ai_beskrivning_label = Label("Ange en beskrivning:")
    ai_beskrivning_input = Textarea(layout=widgets2.Layout(height="100px", width="50%"))
    ai_sok = Button(description="Sök")
    ai_sok.on_click(on_search_click)
    ai = VBox([ai_headline, ai_beskrivning_label, ai_beskrivning_input, ai_sok])
    display(ai)
  with hitta_tb.output_to(0):
    display(df)

with main_tb.output_to(2):
  #headline_eco = HTML("<h1>Dataekosystem</h1>")

  w = GraphWidget(Layout(height='1000px', width='2000px'))
  w.nodes = [
      {"id": 0, "properties": {"label": "Kronofogdemyndighetens datahubb", "url": "https://github.com/kronofogdemyndigheten/datahubb/datahubbsdefinition.json","origin":"1"}},
  #    {"id": 1, "properties": {"label": "Lantmäteriets datalabb", "url": "https://github.com/lantmateriet/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 2, "properties": {"label": "eSams datahubb", "url": "https://github.com/esamverka/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 3, "properties": {"label": "ENA byggblockshubb", "url": "https://github.com/ENA/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 4, "properties": {"label": "Försäkringskassans datahubb", "url": "https://github.com/forsakringskassan/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 5, "properties": {"label": "Bolagsverket", "url": "https://github.com/bolagsverket/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 6, "properties": {"label": "Trafikverket", "url": "https://github.com/trafikverket/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 7, "properties": {"label": "Försäkringskassan", "url": "https://github.com/forsakringskassan/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 8, "properties": {"label": "AFs jobtechdev", "url": "https://github.com/arbetformedlingen/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 9, "properties": {"label": "Statistiska Centralbyråns datahubb", "url": "https://github.com/statisticssweden/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 10, "properties": {"label": "Riksbankshubben", "url": "https://github.com/riksbanken/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 11, "properties": {"label": "Socialstyrelsens hubb", "url": "https://github.com/socialstyrelsen/datahubb/datahubbsdefinition.json","origin":"0"}},
  #    {"id": 15, "properties": {"label": "National Geodata Platform", "url": "https://github.com/geodataplattformen/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 13, "properties": {"label": "European Statistical System Dataplatform", "url": "https://github.com/ess/datahubb/datahubbsdefinition.json","origin":"0"}},
  #    {"id": 14, "properties": {"label": "European space agency", "url": "https://github.com/ESA/datahubb/datahubbsdefinition.json","origin":"0"}},
  #    {"id": 17, "properties": {"label": "EuroGEO", "url": "https://github.com/EuroGEO/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 16, "properties": {"label": "UN global data platform", "url": "https://github.com/UNGDP/datahubb/datahubbsdefinition.json","origin":"0"}},
      {"id": 12, "properties": {"label": "Riksbankshubben", "url": "https://github.com/riksbanken/datahubb/datahubbsdefinition.json","origin":"0"}}
  ]

  w.edges = [
      {"id": 0, "start": 0, "end": 8, "properties": {}},
      {"id": 17, "start": 16, "end": 13, "properties": {}},
      {"id": 1, "start": 0, "end": 9, "properties": {}},
      {"id": 2, "start": 9, "end": 2, "properties": {}},
      {"id": 3, "start": 2, "end": 3, "properties": {}},
      {"id": 4, "start": 2, "end": 4, "properties": {}},
      {"id": 5, "start": 2, "end": 5, "properties": {}},
      {"id": 6, "start": 2, "end": 6, "properties": {}},
      {"id": 7, "start": 2, "end": 7, "properties": {}},
      {"id": 8, "start": 2, "end": 8, "properties": {}},
      {"id": 17, "start": 8, "end": 3, "properties": {}},
      {"id": 9, "start": 2, "end": 11, "properties": {}},
      {"id": 18, "start": 0, "end": 5, "properties": {}},
      {"id": 10, "start": 9, "end": 11, "properties": {}},
      {"id": 11, "start": 9, "end": 13, "properties": {}},
  #    {"id": 12, "start": 1, "end": 2, "properties": {}},
  #    {"id": 20, "start": 1, "end": 3, "properties": {}},
  #    {"id": 21, "start": 1, "end": 0, "properties": {}},
  #    {"id": 22, "start": 1, "end": 6, "properties": {}},
  #    {"id": 23, "start": 1, "end": 9, "properties": {}},
  #    {"id": 24, "start": 1, "end": 15, "properties": {}},
  #    {"id": 19, "start": 1, "end": 17, "properties": {}},
  #    {"id": 13, "start": 1, "end": 14, "properties": {}},
      {"id": 14, "start": 9, "end": 12, "properties": {}},
      {"id": 16, "start": 9, "end": 10, "properties": {}}
  ]

  styles = {
    "1": {"color":"#6C7400", "shape":"round-rectangle", "label":"iata"},
    "0": {"color":"#005977", "shape":"round-rectangle", "label":"name"}
  }  
  w.set_node_styles_mapping(lambda index, node : styles.get(node["properties"]["origin"], {}))
  w.set_graph_layout('radial')
  w.set_graph_layout('organic')
  #w.graph_layout = "organic_edge_router"

  #form_eco = VBox([headline_eco, w])
  display(w)

with main_tb.output_to(1):
  skapa_tb = widgets.TabBar(['Information om datahubb', 'Lägg till infomation om annan datahub', 'Artefakter'])
  with skapa_tb.output_to(2):
    headline3 = HTML(value="<h2>Artefakter</h2>")
    benamning_label3 = Label("Benämning:")
    benamning_input3 = Text()
    beskrivning_label = Label("Beskrivning:")
    beskrivning_input = Text()
    licens_label = Label("Licens:")
    licens_input = Text()
    tillganglighetsgrad_label = Label("Tillgänglighetsgrad:")
    tillganglighetsgrad_input = Text()
    kontaktvag_label = Label("Kontaktväg:")
    kontaktvag_input = Text()
    nyckelord_label = Label("Nyckelord:")
    nyckelord_input = Text()
    skapat_datum_label = Label("Skapat datum:")
    skapat_datum_input = Text()
    beroenden_label = Label("Beroenden:")
    beroenden_input = Text()
    sokvag_label = Label("Sökväg:")
    sokvag_input = Text()
    lagg_till_button3 = Button(description="Lägg till")
    # Attach the add_artifact function to the "lagg_till_button3" button click event
    lagg_till_button3.on_click(add_artifact)

    form3 = VBox([headline3, benamning_label3, benamning_input3, beskrivning_label, beskrivning_input, licens_label, licens_input, tillganglighetsgrad_label, tillganglighetsgrad_input, kontaktvag_label, kontaktvag_input, nyckelord_label, nyckelord_input, skapat_datum_label, skapat_datum_input, beroenden_label, beroenden_input, sokvag_label, sokvag_input, lagg_till_button3 ])
    display(form3)

  with skapa_tb.output_to(1):
    headline2 = HTML(value="<h2>Lägg till information om annan datahubb</h2>")
    benamning_label2 = Label("Benämning:")
    benamning_input2 = Text()
    sokvag_till_definition_label = Label("Sökväg till definition:")
    sokvag_till_definition_input = Text()
    sokvag_till_mer_info_label = Label("Sökväg till mer information om datahubben:")
    sokvag_till_mer_info_input = Text()
    lagg_till_button2 = Button(description="Lägg till")
    # Attach the add_connected_datahub function to the "lagg_till_button2" button click event
    lagg_till_button2.on_click(add_connected_datahub)

    connected_datahub_labels = VBox([])
    form2 = VBox([headline2, benamning_label2, benamning_input2, sokvag_till_definition_label, sokvag_till_definition_input, sokvag_till_mer_info_label, sokvag_till_mer_info_input, lagg_till_button2, connected_datahub_labels])
    display(form2)

  with skapa_tb.output_to(0):
    headline1 = HTML("<h1>Information om datahubben</h1>")
    benamning_label1 = Label("Benämning:")
    benamning_input1 = Text()
    organisatorisk_tillhorighet_label = Label("Organisatorisk tillhörighet:")
    organisatorisk_tillhorighet_input = Text()
    sparahub = Button(description="Spara")
    
    # Attach the save_datadefinition function to the button's click event
    sparahub.on_click(save_datadefinition)


    form1 = VBox([headline1, benamning_label1, benamning_input1, organisatorisk_tillhorighet_label, organisatorisk_tillhorighet_input, sparahub])
    display(form1)

with main_tb.output_to(0):
  md_text = markdown.markdown("""
  <h1>Introduktion</h1>
  
  * Här kan du lägga till information om och artefakter i din datahubb för att dela med dig av....

  <h1>Nyheter i dataekosystemet:</h1>
  Sedan du senast besökte denna sida har dessa datahubbar lagts till i ekosystemet:

  * Lantmäteriets datahubb
  * AI Sweden

  <h2>Nya artefakter kopplat till dina tidigare sökningar:</h2>

  * SCB - Guide för tillgängliggörande av öppna data enligt licensmodell CC0
  * eSam - Vägledning för pseudonymisering
  * AI Sweden - ML-modell för pseudonymisering
  """)
  display(HTML(value=md_text))

# Create a VBox to hold the connected datahub labels
connected_datahub_labels = VBox([])

# Create a VBox to hold the artifact labels
artifact_labels = VBox([])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

HTML(value='<h1>Introduktion</h1>\n\n<ul>\n<li>Här\xa0kan\xa0du\xa0lägga\xa0till\xa0information\xa0om\xa0och\x…

<IPython.core.display.Javascript object>